# HEADER

In [ ]:
# create SQL table in the traditional way

import sqlite3
conn = sqlite3.connect('restaurantmenu.db')

## PROCESSS 






## SQLite PROCESSS 

1. connect to databse  
2. get the cursor - cur = con.cursor() 
3. execute sql 
- create
- query data (SELECT)
   - row cursor   
         - fetchone --> row
         - fetchall() --> rows
         - If there is no more data left, it returns None
   - dictionary cursor --> con.row_factory = lite.Row
          - fetchall()
              - use forloop and get by "key" - row["Id"], row["Name"], row["Price"]
          - fetchone 
          
    - add data (INSERT INTO)
4. commit

5. UPDATE
6. DELETE
7. Parameterized queries - We use placeholders instead of directly writing the values into the statements. Parameterized queries increase security and   performance.
    - "UPDATE Cars SET Price=? WHERE Id=?", (uPrice, uId)
    - "SELECT Name, Price FROM Cars WHERE Id=:Id", {"Id": uId}


8. Export and import of data


## create SQL table in the traditional way

In [ ]:
conn.execute( '''CREATE TABLE restaurant (id INTEGER PRIMARY KEY ASC, name varchar(250) NOT NULL)''')

In [ ]:
conn.execute( '''CREATE TABLE menu_item (id INTEGER PRIMARY KEY ASC, name varchar(250), description varchar(250) NOT NULL, \ 
restaurant_id INTEGER NOT NULL, FOREIGN KEY(restaurant_id) REFERENCES restaurant(id))''')

In [ ]:
conn.commit()

In [ ]:
conn.close()

## setup database/table in the ORM way

In [2]:
# 1. configuration
import sys
from sqlalchemy import Column, ForeignKey, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship #foreigh key relatonship
from sqlalchemy import create_engine
Base = declarative_base()

# 2. create class and table representation

class Restaurant(Base):#extends the Base class
    
    __tablename__ = 'restaurant' #table representation in database
    
    #3. mapper code - column<->py variable 
    id = Column(Integer, primary_key = True)
    name = Column(String(80),nullable = False)
    
    
    
# 2. create class and table representation  

class MenuItem(Base):
    
    __tablename__ = 'menu_item'
    
    #3. mapper code - column<->py variable
    name = Column(String(250),nullable = False)
    id = Column(Integer,primary_key = True)
    course = Column(String(250))
    description = Column(String(250))
    price = Column(Float)
    restaurant_id = Column(Integer, ForeignKey('restaurant.id')) #look inside the restaurant table and retrive its \
    #id col whenever i ask restaurant_id 
    restaurant = relationship(Restaurant)

    
## END
engine = create_engine('sqlite:///restaurantmenu_orm.db',echo=True)
Base.metadata.create_all(engine)

2016-06-21 22:21:10,370 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-06-21 22:21:10,373 INFO sqlalchemy.engine.base.Engine ()
2016-06-21 22:21:10,375 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-06-21 22:21:10,377 INFO sqlalchemy.engine.base.Engine ()
2016-06-21 22:21:10,379 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("menu_item")
2016-06-21 22:21:10,381 INFO sqlalchemy.engine.base.Engine ()
2016-06-21 22:21:10,383 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("restaurant")
2016-06-21 22:21:10,385 INFO sqlalchemy.engine.base.Engine ()
2016-06-21 22:21:10,387 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE restaurant (
	id INTEGER NOT NULL, 
	name VARCHAR(80) NOT NULL, 
	PRIMARY KEY (id)
)


2016-06-21 22:21:10,396 INFO sqlalchemy.engine.base.Engine ()
2016-06-21 22:21:10,412 INFO sqlalchemy.engine.base.Engine COMMIT
2016-06-21 22:21:10,414 INFO sqlalchemy.engin

## populate (insert) to create table 

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker #like cursor

In [2]:
from database_setup import Base, Restaurant, MenuItem
engine = create_engine('sqlite:///restaurantmenu_orm.db')

In [3]:
Base.metadata.bind = engine #make connection between class and actual table

In [4]:
DBSession = sessionmaker(bind = engine) #sqlalchemy execute database operation via vm interface called session 
# a seesion allows us to write down all the command to execute, but it works until we commit

In [5]:
session = DBSession() #session is actual a stage zone for all of the object loaded into db

In [6]:
# add my restaurant 

myFirstRestaurant = Restaurant(name = '比薩大王')
mySecondRestaurant = Restaurant(name = '永和豆漿')

In [8]:
session.add(myFirstRestaurant)
session.add(mySecondRestaurant)

In [9]:
session.commit()

In [10]:
session.query(Restaurant).all() #find the corresponding table in db and return it as list in python - 0x1c17695e0f0 

In [11]:
cheespizza = MenuItem(name = "Cheese Pizza", description="以天然原料製成 加入很多Mozzarella cheese", course = "Entree", price = 150, \
restaurant = myFirstRestaurant)
soymilk = MenuItem(name = "soymilk", description="美味豆漿以天然原料製成", course = "Beverage", price = 90, \
restaurant = mySecondRestaurant)

In [12]:
session.add(cheespizza)
session.add(soymilk)

In [13]:
session.commit()

In [14]:
session.query(MenuItem).all()

## CRUD read   

In [19]:
firstResult = session.query(Restaurant).first()

In [20]:
firstResult.name

'比薩大王'

In [21]:
firstResult.id

1

#### Run addmoremenus_second_version.py to populate more menu item 

In [22]:
session.query(Restaurant).all()

In [23]:
MenuItems = session.query(MenuItem).all()

In [39]:
for item in MenuItems:
    print(item.restaurant.name)
    print("menu:%s, price:%s, course:%s"%(item.name,item.price,item.course))
    print("description: %s"% item.description)

Urban Burger
menu:Veggie Burger, price:7.5, course:Entree
description: Juicy grilled veggie patty with tomato mayo and lettuce
Urban Burger
menu:French Fries, price:2.49, course:Appetizer
description: with garlic and parmesan
Urban Burger
menu:Chicken Burger, price:5.5, course:Entree
description: Juicy grilled chicken patty with tomato mayo and lettuce
Urban Burger
menu:Chocolate Cake, price:3.99, course:Dessert
description: fresh baked and served with ice cream
Urban Burger
menu:Sirloin Burger, price:7.99, course:Entree
description: Made with grade A beef
Urban Burger
menu:Root Beer, price:1.99, course:Beverage
description: 16oz of refreshing goodness
Urban Burger
menu:Iced Tea, price:0.99, course:Beverage
description: with Lemon
Urban Burger
menu:Grilled Cheese Sandwich, price:3.49, course:Entree
description: On texas toast with American Cheese
Urban Burger
menu:Veggie Burger, price:5.99, course:Entree
description: Made with freshest of ingredients and home grown spices
Super Stir Fr

### 配合query的幾個常用function  
#### you can refer to http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#querying

In [24]:
MenuItemResults = session.query(MenuItem)
MenuItemResults.count()

55

In [25]:
menuitems= session.query(MenuItem).filter(MenuItem.course=="Entree").all()
for m in menuitems:
    print(m.name)

Cheese Pizza
Veggie Burger
Chicken Burger
Sirloin Burger
Grilled Cheese Sandwich
Veggie Burger
Chicken Stir Fry
Peking Duck
Spicy Tuna Roll
Nepali Momo 
Beef Noodle Soup
Ramen
Pho
Gyoza
Stinky Tofu
Veggie Burger
Mushroom risotto
Aloo Gobi Burrito
Veggie Burger
Shellfish Tower
Chicken and Rice
Mom's Spaghetti
Tonkatsu Ramen
Lamb Curry
Chicken Marsala
Veggie Burger
Chicken Fried Steak
Broiled salmon
Tandoori Chicken
Veggie Burger
Super Burrito Al Pastor
Cachapa
皮蛋擂辣椒


In [26]:
RestaurantResult = session.query(Restaurant)
RestaurantResult.count()
type(RestaurantResult[0].name)

str

### exercise2 - Retrieve what "Urban Burger" and 'Super Stir Fry'

In [46]:

tagret_restaurants = session.query(Restaurant).filter(Restaurant.name.in_(['Urban Burger','Super Stir Fry']))
for res in tagret_restaurants:
    print("Restaurant: ",res.name)
    tagret_restaurant_menuitems = session.query(MenuItem).filter(MenuItem.restaurant == res).all()
    for menu in tagret_restaurant_menuitems:
            print("     menuitem: %s price: %s"%(menu.name, menu.price))



Restaurant:  Urban Burger
     menuitem: Veggie Burger price: 7.5
     menuitem: French Fries price: 2.49
     menuitem: Chicken Burger price: 5.5
     menuitem: Chocolate Cake price: 3.99
     menuitem: Sirloin Burger price: 7.99
     menuitem: Root Beer price: 1.99
     menuitem: Iced Tea price: 0.99
     menuitem: Grilled Cheese Sandwich price: 3.49
     menuitem: Veggie Burger price: 5.99
Restaurant:  Super Stir Fry
     menuitem: Chicken Stir Fry price: 7.99
     menuitem: Peking Duck price: 25.0
     menuitem: Spicy Tuna Roll price: 15.0
     menuitem: Nepali Momo  price: 12.0
     menuitem: Beef Noodle Soup price: 14.0
     menuitem: Ramen price: 12.0


### exercise2 - Retrieve the offer from non-english restaurants

In [29]:
def isEnglish(s):
    try:
        s.encode('ascii')
    except UnicodeEncodeError:
        return False
    else:
        return True

all_restaurants = session.query(Restaurant)#here 's the query objects, not the list
non_english_restaurants = [res for res in all_restaurants if not isEnglish(res.name)]
for res in non_english_restaurants:
    print("Restaurant: ",res.name)
    non_english_restaurants_menuitems = session.query(MenuItem).filter(MenuItem.restaurant == res).all()
    for menu in non_english_restaurants_menuitems:
        print("     menuitem:", menu.name)


    

Restaurant:  比薩大王
     menuitem: Cheese Pizza
Restaurant:  大宅門
     menuitem: 特製酸梅湯
     menuitem: 皮蛋擂辣椒


### exercise2 - Retrieve the restaurants offering 'French Fries' with price smaller than 3 and their description and price 

In [18]:
target_menuitems = session.query(MenuItem).filter(MenuItem.name == "French Fries").filter(MenuItem.price < 3)

In [26]:
for item in target_menuitems:
    print("Restaurant: ",item.restaurant.name)
    print("%s and price: %s" % (item.name, item.price) )

Restaurant:  Urban Burger
French Fries and price: 2.49
Restaurant:  State Bird Provisions
French Fries and price: 2.83


## UPDATE

In [7]:
veggieBurgers = session.query(MenuItem).filter(MenuItem.name == 'Veggie Burger') #return query object 
type(veggieBurgers)

sqlalchemy.orm.query.Query

In [8]:
veggieBurgers = session.query(MenuItem).filter_by(name = 'Veggie Burger')

In [9]:
for veggieBurger in veggieBurgers:
    print(veggieBurger.id, veggieBurger.price, veggieBurger.restaurant.name, sep=" ; ")


3 ; 7.5 ; Urban Burger
11 ; 5.99 ; Urban Burger
22 ; 9.5 ; Panda Garden
29 ; 6.8 ; Thyme for That Vegetarian Cuisine 
40 ; 3.95 ; Andala's
46 ; 2.95 ; Auntie Ann's Diner' 


In [11]:
PandaVeggieBurger = session.query(MenuItem).filter_by(id = 21).one() #return the panda garden's veggie bugger class object 



In [12]:
PandaVeggieBurger.price

'6.99'

In [13]:
PandaVeggieBurger.price = 2.99

In [14]:
session.add(PandaVeggieBurger)

In [15]:
session.commit()

## CRUD DELETE

In [16]:
spinachIceCream = session.query(MenuItem).filter_by(name = 'Spinach Ice Cream').one()

In [17]:
type(spinachIceCream)

database_setup.MenuItem

In [18]:
spinachIceCream = session.query(MenuItem).filter(MenuItem.name == 'Spinach Ice Cream').one()

In [19]:
type(spinachIceCream)

database_setup.MenuItem

In [20]:
spinachIceCream.restaurant.name

"Auntie Ann's Diner' "

In [21]:
session.delete(spinachIceCream)

In [22]:
session.commit()

In [23]:
spinachIceCream = session.query(MenuItem).filter(MenuItem.name == 'Spinach Ice Cream').one()

NoResultFound: No row was found for one()